In [1]:
import random
import numpy as np

from RobotEnviroment.init import *
from RobotEnviroment.arenas import RectangularArena
from PointClouds.PushFinder import *
from PointClouds.Registration import *
from PathPlanning.scanning import *
from PathPlanning.motion import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Initialise Robot and Configuration

In [2]:
ON_REAL = False
INITIAL_OBJ_POS = np.array([-.50, -.1, .69])

C = setup_config(INITIAL_OBJ_POS, ON_REAL)
bot = startup_robot(C, ON_REAL)

Define an arena to stop the robot from moving the object too far

In [3]:
TABLE_CENTER = np.array([-.23, -.16, .651])
TABLE_DIMS = np.array([.89, .55])
ROBOT_POS = np.array([-.03, -.24, .651])
ROBOT_RING = .29

arena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0], height=TABLE_DIMS[1], middlePCirc=ROBOT_POS, innerR=ROBOT_RING)
arena.plotArena(C)

### Main pushing loop
- Look towards the predicted object position
- Get the object's point cloud and calculate its normals
- Choose a random pushing point in the point cloud
- Calculate the pushing motion
- Push if the motion is feasible

In [4]:
NUMBER_OF_PUSH_TRIALS = 10
predObjPos = INITIAL_OBJ_POS

pointClouds = [] # Stores dicts {"world_position": [x, y, z], "pc": np.array([])}
minNumScans = 2
fullPC = np.array([])

for i in range(NUMBER_OF_PUSH_TRIALS):

    print("Starting Trial Number ", i+1)

    # Locate and scan object
    # lookAtObj(predObjPos, bot, C)
    lookAtObjRandAngle(predObjPos, bot, C,)
    predObjPos, pointCloud = getScannedObject(bot, C, arena)
    if not len(predObjPos):
        print ("Lost the Object!")
        break
    push_points  = getPushPoints(pointCloud, verbose=1)

    # Store its point cloud
    for i, _ in enumerate(pointCloud):
        pointCloud[i] -= predObjPos
    pointClouds.append(pointCloud)

    if len(pointClouds) >= minNumScans:
        fullPC = joinOffsetPCS(pointClouds.copy(), verbose=0)

    for i in range(5): # This counts the number of attempts to push an object from this angle. If it reaches 5 we try a new angle
        if not len(push_points):
            print("No Viable Push Points!")
            break

        pushP = random.choice(push_points)
        # push_points.remove(pushP)

        waypoints = pushPCpoint(pushP[0], pushP[1], predObjPos, config=C)

        path, feasible = computeKomo(C, bot, waypoints, verbose=2)
        if feasible:
            print("+------------------------+")
            print("|    Pushing Object...   |")
            print("+------------------------+")
            bot.move(path, [4.])
            while bot.getTimeToEnd() > 0:
                # y, J = C.eval(ry.FS.position, ["l_gripper"], [[1,0,0]])
                # bot.setCompliance(J, 1.)
                # print(' direct:', J @ bot.get_tauExternal(),
                #     ' pseudoInv:', np.linalg.pinv(J.T, rcond=1e-3) @ bot.get_tauExternal())
                bot.sync(C, .1)
            break
        else:
            print("+------------------------+")
            print("| Point is not Feasible! |")
            print("+------------------------+")

Starting Trial Number  1
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'way0'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'way1'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'way2'
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'way3'
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals:  12  A(x):    71.8411  f:    19.2391  g:          0  h:    17.2111  |x-x'|:   0.650191 	stop:DeltaConverge
==nlp== it:   1  evals:  12  A(x):    387.454  mu:5
==nlp== it:   1  evals:  30  A(x):     261.16  f:    106.808  g:          0  h:    12.0536  |x-x'|:    0.30337 	stop:DeltaConverge
==nlp== it:   2  evals:  30  A(x):    863.849  mu:25
==nlp== it:   2  evals:  51  A(x):    700.677  f:    261.357  g:          0  h:    9.00887  |x-x'|:    1.03689 	stop:DeltaConverge
==nlp== it:   3  evals:  51  A(x):    2362.97  mu:1

KeyboardInterrupt: 

In [ ]:
bot.home(C)

In [ ]:
del bot
del C